# **실습 3-2 : Char-RNN**



## **Import Module**

In [1]:
%tensorflow_version 2.x

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

tf.__version__

TensorFlow 2.x selected.


'2.0.0'

## **DataSet**

### 학습할 문장 만들기

In [2]:
# 학습할 문장
sentence = ( "Rhyme, alliteration, assonance and consonance are ways of creating repetitive patterns of sound. They may be used as an independent structural element in a poem, to reinforce rhythmic patterns, or as an ornamental element.[66] They can also carry a meaning separate from the repetitive sound patterns created. For example, Chaucer used heavy alliteration to mock Old English verse and to paint a character as archaic.[67]"

"Rhyme consists of identical (hard-rhyme) or similar (soft-rhyme) sounds placed at the ends of lines or at predictable locations within lines (internal rhyme). Languages vary in the richness of their rhyming structures; Italian, for example, has a rich rhyming structure permitting maintenance of a limited set of rhymes throughout a lengthy poem. The richness results from word endings that follow regular forms. English, with its irregular word endings adopted from other languages, is less rich in rhyme.[68] The degree of richness of a languages rhyming structures plays a substantial role in determining what poetic forms are commonly used in that language.[69]"

"Alliteration is the repetition of letters or letter-sounds at the beginning of two or more words immediately succeeding each other, or at short intervals; or the recurrence of the same letter in accented parts of words. Alliteration and assonance played a key role in structuring early Germanic, Norse and Old English forms of poetry. The alliterative patterns of early Germanic poetry interweave meter and alliteration as a key part of their structure, so that the metrical pattern determines when the listener expects instances of alliteration to occur. This can be compared to an ornamental use of alliteration in most Modern European poetry, where alliterative patterns are not formal or carried through full stanzas. Alliteration is particularly useful in languages with less rich rhyming structures."

"Assonance, where the use of similar vowel sounds within a word rather than similar sounds at the beginning or end of a word, was widely used in skaldic poetry but goes back to the Homeric epic.[70] Because verbs carry much of the pitch in the English language, assonance can loosely evoke the tonal elements of Chinese poetry and so is useful in translating Chinese poetry.[71] Consonance occurs where a consonant sound is repeated throughout a sentence without putting the sound only at the front of a word. Consonance provokes a more subtle effect than alliteration and so is less useful as a structural element.[69]"
)
# sentence = ("if you want to build a ship, don't drum up people together to "
#             "collect wood and don't assign them tasks and work, but rather "
#             "teach them to long for the endless immensity of the sea." )
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)
print ("Length of 'test sentence' is %s" %len(sentence))

FOLLOWING IS OUR TRAINING SEQUENCE:
Rhyme, alliteration, assonance and consonance are ways of creating repetitive patterns of sound. They may be used as an independent structural element in a poem, to reinforce rhythmic patterns, or as an ornamental element.[66] They can also carry a meaning separate from the repetitive sound patterns created. For example, Chaucer used heavy alliteration to mock Old English verse and to paint a character as archaic.[67]Rhyme consists of identical (hard-rhyme) or similar (soft-rhyme) sounds placed at the ends of lines or at predictable locations within lines (internal rhyme). Languages vary in the richness of their rhyming structures; Italian, for example, has a rich rhyming structure permitting maintenance of a limited set of rhymes throughout a lengthy poem. The richness results from word endings that follow regular forms. English, with its irregular word endings adopted from other languages, is less rich in rhyme.[68] The degree of richness of a lang

### 입력 문자열과 타겟 문자 준비

In [3]:
# make charater dictionary 
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
print ("CHARACTERS: ")
print (len(char_set))
print (char_set)
print ("DICTIONARY: ")
print (len(char_dic))
print (char_dic)

CHARACTERS: 
53
['b', 'F', 'R', 'r', 'H', 'f', 'e', '[', 'k', '7', '9', 'M', 'C', 's', 'v', ')', '8', 't', 'x', 'u', 'I', ',', 'B', '1', 'd', 'o', '6', 'l', 'y', '0', 'a', 'G', 'E', 'p', '.', ']', 'N', 'w', 'n', 'A', '-', '(', 'T', 'm', 'h', 'g', ' ', 'i', 'L', 'z', 'O', 'c', ';']
DICTIONARY: 
53
{'b': 0, 'F': 1, 'R': 2, 'r': 3, 'H': 4, 'f': 5, 'e': 6, '[': 7, 'k': 8, '7': 9, '9': 10, 'M': 11, 'C': 12, 's': 13, 'v': 14, ')': 15, '8': 16, 't': 17, 'x': 18, 'u': 19, 'I': 20, ',': 21, 'B': 22, '1': 23, 'd': 24, 'o': 25, '6': 26, 'l': 27, 'y': 28, '0': 29, 'a': 30, 'G': 31, 'E': 32, 'p': 33, '.': 34, ']': 35, 'N': 36, 'w': 37, 'n': 38, 'A': 39, '-': 40, '(': 41, 'T': 42, 'm': 43, 'h': 44, 'g': 45, ' ': 46, 'i': 47, 'L': 48, 'z': 49, 'O': 50, 'c': 51, ';': 52}


In [4]:
data_dim    = len(char_set) # train data X:input
num_classes = len(char_set) # trian data Y:target
sequence_length = 10        # any arbitrary number
print ('data_dim : %d' %data_dim)
print ('num_classes : %d' %num_classes)

data_dim : 53
num_classes : 53


In [5]:
dataX = []  # input sequence list
dataY = []  # target sequence list 

# we will make 170 sequences 
for i in range(0, len(sentence) - sequence_length):
    # 10 characters = 1 training sequence
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1 : i+sequence_length+1]
    # convert x, y str to index(int)
    x_idx = [char_dic[c] for c in x_str] 
    y_idx = [char_dic[c] for c in y_str] 
    # append to dataset list
    dataX.append(x_idx)
    dataY.append(y_idx)

    # monitoring
    if i<5:
        print ("[%3d/%3d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*10, x_idx, y_idx))

[  0/2509] [Rhyme, all]=>[hyme, alli]
          [2, 44, 28, 43, 6, 21, 46, 30, 27, 27]=>[44, 28, 43, 6, 21, 46, 30, 27, 27, 47]
[  1/2509] [hyme, alli]=>[yme, allit]
          [44, 28, 43, 6, 21, 46, 30, 27, 27, 47]=>[28, 43, 6, 21, 46, 30, 27, 27, 47, 17]
[  2/2509] [yme, allit]=>[me, allite]
          [28, 43, 6, 21, 46, 30, 27, 27, 47, 17]=>[43, 6, 21, 46, 30, 27, 27, 47, 17, 6]
[  3/2509] [me, allite]=>[e, alliter]
          [43, 6, 21, 46, 30, 27, 27, 47, 17, 6]=>[6, 21, 46, 30, 27, 27, 47, 17, 6, 3]
[  4/2509] [e, alliter]=>[, allitera]
          [6, 21, 46, 30, 27, 27, 47, 17, 6, 3]=>[21, 46, 30, 27, 27, 47, 17, 6, 3, 30]


### 입력문자 list를 3-dim로, 타겟문자 list를 2-dim로 변환  

In [6]:
# input tensor 생성 
X = np.array(dataX)                                      # ndarray(170,10)<-[170,10]
# convert list sequence to 3dim array(one-hot coding)
sequences = [tf.keras.utils.to_categorical(
                 x, num_classes = data_dim) for x in X]  # list[170,10,25]
X = np.array(sequences)                                  # ndarray(170,10,25)

# Target tensor 생성
#y = np.array(dataY)[:,-1]                                # ndarray(170,)<-[170,10]
y = np.array(dataY)                                       # ndarray(170,10)<-[170,10]
# 만약 :. -1로 끊어진 경우에는 하나의 캐릭터를 빼내고 그걸로 학습. 근데 전체로 할때는 한 캐릭터 뿐만 아니라 중간 값도 맞기를 원하기 때문에 더 좋은 결과가 나온다.
# convert the vector to 2-dim 
y = tf.keras.utils.to_categorical( 
                  y, num_classes = data_dim)             # (170,25) -> (170,10,25)

print (y.shape) # (170,10,25)
#print (y[0])    # t -> one-hot ,  (if you wan't')


# 마지막 캐릭터 하나만 비교하는 것이 아니라 캐릭터 전체를 비교해줄 필ㅇ요가 있고, 

(2499, 10, 53)


## **Model**

### Define

In [7]:
# Sequenctial model define: LSTM + Dense
hidden_dim = 150
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(hidden_dim,
          input_shape=(sequence_length,num_classes),
          return_sequences=True)) # (bs,10,hidden_dim)<-(bs,hidden_dim) 시퀀스에서까지의 분포를 다 학습한다. 처음의 코드는 전체적으로 볼 때 하나의 코드였ㄷ고
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LSTM(hidden_dim, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_dim, return_sequences=True))
model.add(tf.keras.layers.Dense(data_dim, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 150)           122400    
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 150)           600       
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 150)           180600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 150)           180600    
_________________________________________________________________
dense (Dense)                (None, 10, 53)            8003      
Total params: 492,203
Trainable params: 491,903
Non-trainable params: 300
_________________________________________________________________


### Compile

In [0]:
model.compile(loss ='categorical_crossentropy', # one-hot coding
              optimizer='adam', 
              metrics=['accuracy'])

### Fit

Epoch 1000/1000   
170/170 [==============================] - 0s 235us/sample - loss: 0.0012 - accuracy: 1.0000    
CPU times: user 48.3 s, sys: 6.54 s, total: 54.8 s    
Wall time: 47.8 s (@Notebook Setting/GPU)

In [9]:
%%time
model.fit(X,            # X.shape : (170, 10, 25)
          y,            # y.shape : (170, 10, 25)
          epochs=400)

Train on 2499 samples
Epoch 1/400
2499/2499 [==============================] - 11s 4ms/sample - loss: 3.0828 - accuracy: 0.1632
Epoch 2/400
2499/2499 [==============================] - 2s 750us/sample - loss: 2.4862 - accuracy: 0.2956
Epoch 3/400
2499/2499 [==============================] - 2s 741us/sample - loss: 1.9736 - accuracy: 0.4386
Epoch 4/400
2499/2499 [==============================] - 2s 746us/sample - loss: 1.6152 - accuracy: 0.5378
Epoch 5/400
2499/2499 [==============================] - 2s 747us/sample - loss: 1.3566 - accuracy: 0.6070
Epoch 6/400
2499/2499 [==============================] - 2s 739us/sample - loss: 1.1646 - accuracy: 0.6613
Epoch 7/400
2499/2499 [==============================] - 2s 749us/sample - loss: 1.0170 - accuracy: 0.7057
Epoch 8/400
2499/2499 [==============================] - 2s 752us/sample - loss: 0.9044 - accuracy: 0.7348
Epoch 9/400
2499/2499 [==============================] - 2s 740us/sample - loss: 0.8218 - accuracy: 0.7535
Epoch 10/400
249

KeyboardInterrupt: ignored

## **Generate Text:** analysis

### Related Module Import

### Define generate_seq function

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, char_dic, seq_length, seed_text, n_chars):
	in_text = seed_text									# seed text + generated text
	# generate a fixed number of characters
	for i in range(n_chars):
		# encode the characters as integers
		encoded = [char_dic[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		encoded = tf.keras.utils.to_categorical(encoded, num_classes=len(char_dic))
		#encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		for char, index in char_dic.items():
			if index == yhat[0][-1]:
				break
		# append to input
		in_text += char
	return in_text

### Generate a sequence of characters

학습문장: if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

In [17]:
# test start of rhyme
print('Trigger characters: "123 "\nResult: \n"{}"'.format( 
      generate_seq(model, char_dic, sequence_length, 'aaaaaaaaaaaaaaaaaaaaaaa', 200)))
# test mid-line
print('\nTrigger characters: "d"\nResult: \n"{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'd', 200)))
# test not in original
print('\nTrigger characters: "aabbcr than"\nResult: "{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'hi', 200)))
print('\nTrigger characters: "the recurrence of the same lette"\nResult: "{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'the recurrence of the same lette', 200)))

print('\nTrigger characters: "r at predic"\nResult: "{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'r at predic', 800)))

Trigger characters: "123 "
Result: 
"aaaaaaaaaaaaaaaaaaaaaaanennds of lines or at predictable locations within lines (internal rhyme). Languages vary in the richness of their structures plays a substantial role in determining what poetic forms are commonly use"

Trigger characters: "d"
Result: 
"dcokeary used in that language.[69]Alliteration is particularly useful in translating Chinese poetry and so is useful in translating Chinese poetry and so is useful in translating Chinese poetry and so"

Trigger characters: "aabbcr than"
Result: "hico the recurrence of the same letter in accented parts of words. Alliteration is particularly useful in translating Chinese poetry and so is useful in translating Chinese poetry and so is useful in tr"

Trigger characters: "the recurrence of the same lette"
Result: "the recurrence of the same letter in accented parts of words. Alliteration is particularly useful in translating Chinese poetry and so is useful in translating Chinese poetry and so is us

In [0]:
#@title
print('\nTrigger characters: "aabbcr than"\nResult: "{}"'.format(
      generate_seq(model, char_dic, sequence_length, 'ports', 200)))

## **실습과제**

### 과제 1. 학습문장을 Keras Documents에서 2000자 정도 가져다 넣어서 실행해 보자('.'뒤에'\n'도 넣어주자) 
#### -- accuracy가 적당한가 확인하자.
#### -- 모델을 강화 하기 위해 `hidden_dim ='값을 150으로 높히고, 
#### -- `#`로 막혀 있는 `BatchNormalization()`을 동작시켜 보자
#### -- `Trigger characters`를 다양하게 바꿔보자 
#### -- 결과를 확인하고, 분석해 보자:
 


위에 있는 것들을 다 종합해본 결과 accuracy의 경우에는 약 80 퍼센트 선에서 진동하는 것을 확인할 수 있었다.

관찰결과 짧은 단어나 낱개의 알파벳의 경우에는 비교적 정상적으로 문장을 출력하는 것을 확인할 수 있었다. 

aaaaaaaaaaaa와 같은 이상한 시작의 경우에는 처음 단어는 제대로 뽑아내지 못했으나 그 이후부터는 비교적 제대로 뽑아내는 것을 확인할 수 있었다.

다만 hi로 시작하는 문장의 경우에는 이상한 점을 발견할 수 있었는데, 후반부에 계속해서 같은 문장이 반복하는 것이었다. 

이는 가장 마지막에 문장의 중간 부분을 테스트한 경우에도 발견됐는데, 이러한 문제점이 공통적으로 발견되는 지점은 so is useful in 이라는 문장이었다.

추정하건데, useful in 이라는 맥락이 등장한 이후에 가장 자주 나온 것이 chinese라는 단어로 시작하는 문장이었고, 학습의 데이터가 제한적이다 보니 useful as와 같은 다른 가능성이 있음에도 가장 높은 가능성을 보이는 chinese 문장이 계속해서 반복적으로 나타난 것이 아닌가 추측해본다.